In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Giải nén file dữ liệu word_embedđing có sẵn:

import zipfile
zip_ref = zipfile.ZipFile('../input/quora-insincere-questions-classification/embeddings.zip', 'r')
zip_ref.printdir()
zip_ref.extract('wiki-news-300d-1M/wiki-news-300d-1M.vec')

- Đọc dữ liệu từu tập csv sau đó chia thành 2 list
+ 1 list chứa những câu insincere
+ 1 list chứa những câu sincere

(Trong dữ liệu ban đầu có tổng số hơn 2 triệu câu, trong đó 92% số câu hỏi được gán nhãn sincere và số còn lại được gán nhãn insicere. Do sự giới hạn của bộ nhớ, ta sẽ lấy 1 số câu với mỗi loại nhãn dán để thực hiện việc huấn luyện và test)

In [ ]:
# ĐỌc dữ liệu và chia thành 2 nhóm sincere và insicere từ tập dữ liệu ban đầu:

df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
insincere_questions = [df['question_text'][idx] for idx in range(len(df)) if df['target'].loc[idx] == 1]
sincere_questions = [df['question_text'][idx] for idx in range(len(df)) if df['target'].loc[idx] == 0]

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

Tạo tập chứa các stop words và các dấu câu (sử dụng trong việc clean dữ liệu)

In [ ]:
stop_words = set(stopwords.words('english'))
puncs = set(string.punctuation)

CLean dữ liệu:
- Xóa các stop words và các dấu câu trong 1 câu
- Chuyển các từ sang kí tự thường

- Chuyển các từ còn lại thành 1 list các từ 

In [ ]:
# Tiến hành chuyển các câu hỉ thành 1 list gồm những từ còn lại sau khi xóa các stop_words và các dấu câu:

def clean(text_list):
    for idx, text in enumerate(text_list):
        text = text.lower().split()
        word_list = [char for char in text if char not in puncs and char not in stop_words]
        text_list[idx] = word_list
    
    return text_list

In [ ]:
insincere_questions = clean(insincere_questions)
sincere_questions = clean(sincere_questions)

- Đọc tập embedding word từ file dữ liệu
(Embedding words là file chứa những vector gồm 300 chiều mã hóa cho 1 từ tiếng anh vd: 'the': '-0.12 0.1678 0.3467 -0.899 ....')
- Sau khi chuyển những từ còn lại trong câu hỏi thành 1 danh sách (đã bỏ những stop word và các dấu câu), ta chuyển những từ đó thành các vector 300 chiều dựa vào tập embedding word

In [ ]:
# Đọc dữ liệu embedding word từ file wiki-news-300d-1M.vec
f = open('./wiki-news-300d-1M/wiki-news-300d-1M.vec')
lines = [line for line in f.readlines()]
lines = lines[2:]
temp = lines[:80000]

In [ ]:
# Lấy 1 số những câu hỏi trong tập huấn luyện để thực hiện cho việc huấn luyện

sample_insincere_questions = insincere_questions[:10000]
sample_sincere_questions = sincere_questions[:10000]
test_insincere_questions = insincere_questions[10000:11000]
test_sincere_questions = sincere_questions[10000:11000]

In [ ]:
#Tạo 1 dictionary embedding_list có dạng 'word': array(300) từ file embedding trong đó array(300) là vector tượng trưng cho các words tương ứng

embedding_list = {}

for val in temp:
    val = val.replace('\n', '').split()
    key = val.pop(0)
    key = key.lower()
    if key in puncs or key in stop_words:
        continue
        
    embedding_list[key] = np.array([float(x) for x in val])

* Viêc clean dữ liệu (xóa 1 số từ) sẽ dẫn đến việc các list từ còn lại có độ dài không giống nhau
=> Khắc phục bằng cách thêm các vector 0 ([0, 0, 0, 0, ...., 0]) vào những từ còn thiếu, từ đó các list trong tập huấn luyện sẽ có cùng cùng độ dài với nhau mà không làm thay đổi ý nghĩa của các câu


In [ ]:
# Tìm độ dài lớn nhất còn lại sau khi đã thực hiện clean dữ liệu 
max_len = 0
for idx in range(len(sincere_questions)):
    if idx < len(insincere_questions):
        if max_len < max(len(sincere_questions[idx]), len(insincere_questions[idx])):
            max_len = max(len(sincere_questions[idx]), len(insincere_questions[idx]))
    else:
        if max_len < len(sincere_questions[idx]):
            max_len = len(sincere_questions[idx])

max_len

In [ ]:
# Tạo vector 0
def generate_mt(key):
    new_arr = np.zeros((key, 300))
    return new_arr

In [ ]:
# Chuyển các từ trong 1 list sang dạng vector đồng thời thực hiện đồng bọ về chiều dài của các list từ trong câu hỏi
# 1 list từ trong dạng vector sẽ có dang: [max_len, 300]
#     * max_len: là số từu có trong list (nếu trong list ban đầu không có đủ số từ, ta thay bằng các vecto 0)
#     * 300: là số chiều vector tượng trưng cho từ đó trong tập word_embedding ở trên


def convert_to_vt(list_questions, max_len):
    for idx in range(len(list_questions)):
        if len(list_questions[idx]) == 0:
            continue
        for i, item in enumerate(list_questions[idx]):
            if item not in embedding_list:
                list_questions[idx][i] = np.zeros(300)
            else:
                list_questions[idx][i] = embedding_list[list_questions[idx][i]]
        if len(list_questions[idx]) < max_len:
            temp_arr = generate_mt(max_len - len(list_questions[idx]))
            list_questions[idx] = np.concatenate((list_questions[idx], temp_arr), axis=0)
            
    return list_questions

In [ ]:
# Chuyển sang vector đối với tập huấn luyện

sample_insincere_questions = convert_to_vt(sample_insincere_questions, max_len)
sample_sincere_questions = convert_to_vt(sample_sincere_questions, max_len)
sample_insincere_questions = [item for item in sample_insincere_questions if type(item) is not list]
sample_sincere_questions = [item for item in sample_sincere_questions if type(item) is not list]

In [ ]:
# Chuyển sang vector đối với tập test

test_insincere_questions = convert_to_vt(test_insincere_questions, max_len)
test_sincere_questions = convert_to_vt(test_sincere_questions, max_len)
test_insincere_questions = [item for item in test_insincere_questions if type(item) is not list]
test_sincere_questions = [item for item in test_sincere_questions if type(item) is not list]

In [ ]:
a = np.array(sample_insincere_questions)
b = np.array(sample_sincere_questions)

In [ ]:
# Sau khi chuyển dữ liệu sang dạng vector, ta nối 2 tập sincere và insincere để tạo thành tập huấn luyện cho mô hình
train_set = np.concatenate((a,b), axis=0)

In [ ]:
train_set = train_set.reshape(train_set.shape[0], -1)

In [ ]:
train_set.shape

In [ ]:
# Tạo tập nhãn dán dựa trên số lượng các câu hỏi của tập huấn luyện
# Tỉ lệ các câu sincere : insincere là 50:50

label1 = np.ones((10000,))
label2 = np.zeros((9999,))
label = np.concatenate((label2, label1), axis=0)

In [ ]:
# Với tập test, ta làm tương tự như các bước đã thực hiện trong tập huấn luyện

a = np.array(test_insincere_questions)
b = np.array(test_sincere_questions)
test_set = np.concatenate((a, b), axis=0)
test_set = test_set.reshape(test_set.shape[0], -1)

label1 = np.ones((1000))
label2 = np.zeros((1000,))
test_label = np.concatenate((label2, label1), axis=0)

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn import metrics
import matplotlib.pyplot as plt

* Sử dụng mô hình LogisticRegression để phân lớp cho bộ dữ liệu có 2 nhãn (sincere:1 và insincere:0)

In [ ]:
# Thực hiện khai báo mô hình và tiến hành huấn luyện (số lần train = 300)
# Sau đó thực hiện đánh giá độ chính xác của tập huấn luyện

clf = LogisticRegression(random_state=0, max_iter=300).fit(train_set, label)
# clf.fit(train_set, label)
clf.score(train_set, label)

In [ ]:
# Tiến hành dự đoán tập test bằng mô hình đã huấn luyện
pred = clf.predict(test_set)

In [ ]:
# Tính độ chính xác của việc dự đoán trên tập test 
cr = 0
for i in range(len(test_set)):
    if pred[i] == test_label[i]:
        cr += 1

print(cr / len(test_set))

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# Điểm F1 của mô hình 
f1 = f1_score(test_label, pred)
f1

In [ ]:
accuracy_score(test_label, pred)

In [ ]:
# Báo cáo kết quả thực hiện:

from sklearn.metrics import classification_report
print(classification_report(test_label, pred))